
# Analysis Comparison between Original and Expanded Lexicons on Cohort Analysis Task

Subreddit: Mental Heatlh

In [1]:
from pathlib import Path
import pandas as pd

from config import global_config
from lexicons2 import Values, ValuesExpanded, Liwc2015, Liwc2015Expanded
from spaces import WordEmbeddings, LabelEmbeddings

In [2]:
COHORT_NAME = 'mentalhealth'

In [3]:
cohort_corpus_path = global_config.reddit.submissions[COHORT_NAME]
cohort_models_path = f'{global_config.paths.models}/{COHORT_NAME}'

control_corpus_path = global_config.reddit.submissions['IAMA']
control_models_path = f'{global_config.paths.models}/IAMA'

In [4]:
cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks.csv'
expanded_cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks_expanded.csv'

control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks.csv'
expanded_control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks_expanded.csv'

relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks.csv'
expanded_relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks_expanded.csv'

Build Control Spaces

In [5]:
control_word_space = WordEmbeddings(corpus_path=control_corpus_path, model_path=control_models_path).build()

In [6]:
org_control_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=control_word_space).build()

In [7]:
expanded_control_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=control_word_space).build()

Build Cohort Spaces

In [8]:
cohort_word_space = WordEmbeddings(corpus_path=cohort_corpus_path, model_path=cohort_models_path).build()

In [9]:
org_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=cohort_word_space).build()

In [10]:
expanded_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=cohort_word_space).build()

Save and view results

In [11]:
def save_if_not_save_and_return(space: LabelEmbeddings, path: str):
	if not Path(path).exists():
		space.save_distances_to_csv(path)
	return pd.read_csv(path, names=['label_one', 'label_two', 'distance'])

control_ranks = save_if_not_save_and_return(org_control_label_space, control_ranks_csv)
expanded_control_ranks = save_if_not_save_and_return(expanded_control_label_space, expanded_control_ranks_csv)

cohort_ranks = save_if_not_save_and_return(org_cohort_label_space, cohort_ranks_csv)
expanded_cohort_ranks = save_if_not_save_and_return(expanded_cohort_label_space, expanded_cohort_ranks_csv)

In [12]:
org_cohort_label_space.compute_rank_deltas(org_control_label_space, relative_ranks_csv)
relative_ranks = pd.read_csv(relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

expanded_cohort_label_space.compute_rank_deltas(expanded_control_label_space, expanded_relative_ranks_csv)
expanded_relative_ranks = pd.read_csv(expanded_relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

View Stuff

In [13]:
relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,values:learning,64,28,0.220263,0.178845,36,0.041419
1,liwc2015:function,values:feeling-good,76,43,0.591848,0.499940,33,0.091908
2,liwc2015:function,values:children,75,50,0.466853,0.680796,25,-0.213943
3,liwc2015:function,liwc2015:money,70,51,0.295679,0.681783,19,-0.386104
4,liwc2015:function,values:animals,81,64,0.801544,0.794112,17,0.007432
...,...,...,...,...,...,...,...,...
6967,values:animals,liwc2015:male,29,82,0.800333,1.117406,-53,-0.317073
6968,values:animals,liwc2015:sad,4,58,0.747120,0.906883,-54,-0.159763
6969,values:animals,liwc2015:home,12,68,0.785926,0.975827,-56,-0.189901
6970,values:animals,values:truth,8,71,0.770242,0.982374,-63,-0.212133


In [14]:
expanded_relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,liwc2015:posemo,53,34,0.066118,0.259349,19,-0.193231
1,liwc2015:function,liwc2015:work,56,39,0.089691,0.336770,17,-0.247079
2,liwc2015:function,liwc2015:money,75,58,0.295679,0.681783,17,-0.386104
3,liwc2015:function,liwc2015:pconcern,70,54,0.213819,0.624154,16,-0.410335
4,liwc2015:function,liwc2015:netspeak,80,68,0.596505,0.781215,12,-0.184709
...,...,...,...,...,...,...,...,...
7135,liwc2015:relig,liwc2015:space,14,68,0.742864,1.013450,-54,-0.270586
7136,liwc2015:relig,liwc2015:percept,10,72,0.731795,1.029727,-62,-0.297931
7137,liwc2015:relig,values:parents,3,71,0.717833,1.022422,-68,-0.304589
7138,liwc2015:relig,liwc2015:friend,1,76,0.707746,1.092038,-75,-0.384292
